<a href="https://colab.research.google.com/github/ryancookd/PurahScanner/blob/main/ZeldaTOTK_Ingredient.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

This is an experimental Zelda ToTK Ingredient Scanner

IMPORTS

In [ ]:
import os
import numpy as np
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import layers

In [ ]:
img_height = height
img_width = width
batch_size = 2

model = keras.Sequential([
    layers.Input((28, 28, 1)),
    layers.Conv2D(16, 3, padding ='same'),
    layers.Conv2D(32, 3, padding='same'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(10),
])

NameError: ignored

Create Training and Validation Sets From Data

In [ ]:
ds_train = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/ZeldaTOTK Ingredients',
    labels='inferred',
    label_mode = "int",
    color_mode = 'grayscale',
    batch_size = batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset = "training",
)

ds_validate = tf.keras.preprocessing.image_dataset_from_directory(
    '/content/drive/MyDrive/ZeldaTOTK Ingredients',
    labels='inferred',
    label_mode = "int",
    color_mode = 'grayscale',
    batch_size = batch_size,
    image_size=(img_height, img_width),
    shuffle=True,
    seed=123,
    validation_split=0.2,
    subset = "validation",
)

Found 20 files belonging to 4 classes.
Using 16 files for training.
Found 20 files belonging to 4 classes.
Using 4 files for validation.


Modeling Time Boys

In [ ]:
def augment(x, y):
  image = tf.image.random_brightness(x, max_delta=0.05)
  return image, y

ds_train = ds_train.map(augment)

model.compile(
    optimizer=keras.optimizers.Adam(),
    loss=[
        keras.losses.SparseCategoricalCrossentropy(from_logits=True),
    ],
    metrics=["accuracy"],
)

model.fit(ds_train, epochs=5, verbose=2)

Epoch 1/5
8/8 - 4s - loss: 90.8205 - accuracy: 0.1875 - 4s/epoch - 484ms/step
Epoch 2/5
8/8 - 0s - loss: 26.1427 - accuracy: 0.5625 - 197ms/epoch - 25ms/step
Epoch 3/5
8/8 - 0s - loss: 11.3670 - accuracy: 0.5625 - 194ms/epoch - 24ms/step
Epoch 4/5
8/8 - 0s - loss: 2.4751 - accuracy: 0.8750 - 98ms/epoch - 12ms/step
Epoch 5/5
8/8 - 0s - loss: 0.2060 - accuracy: 0.9375 - 102ms/epoch - 13ms/step


In [ ]:
model.evaluate(ds_validate)

2/2 [==============================] - 1s 8ms/step - loss: 5.0601 - accuracy: 0.5000


[5.060115814208984, 0.5]

Convert the File to Keras

In [ ]:
# Save the entire model as a `.keras` zip archive.
model.save('my_model.keras')

new_model = tf.keras.models.load_model('my_model.keras')

# Show the model architecture
new_model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 28, 28, 16)        160       
                                                                 
 conv2d_1 (Conv2D)           (None, 28, 28, 32)        4640      
                                                                 
 max_pooling2d (MaxPooling2D  (None, 14, 14, 32)       0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 6272)              0         
                                                                 
 dense (Dense)               (None, 10)                62730     
                                                                 
Total params: 67,530
Trainable params: 67,530
Non-trainable params: 0
____________________________________________________

Convert to tfLite

In [ ]:
# Convert the model.
converter = tf.lite.TFLiteConverter.from_keras_model(model)
tflite_model = converter.convert()

# Save the model.
with open('model.tflite', 'wb') as f:
  f.write(tflite_model)